In [1]:
from astropy.cosmology import FlatLambdaCDM
import numpy as np
from scipy.optimize import golden
from scipy.constants import pi
from astropy import units as u
from astropy.constants import G
from astropy.constants import c as c_light

In [2]:
# Define the cosmology
h = 0.7
H0 = h * 100 * u.km / u.s / u.Mpc
cosmo = FlatLambdaCDM(H0=H0, Om0=0.3)

## NFW profile

defined flexibly with any 2 parameters

In [3]:
def Dds_Ds(zl, zs):
    Dds = cosmo.angular_diameter_distance_z1z2(zl, zs)
    Ds  = cosmo.angular_diameter_distance_z1z2(0 , zs)
    return (Dds / Ds).value

#def dccalc(z=None):
def dvir_calc(z=None):
    """Calculate the virial overdensity (instead of 200)."""
    if z is None:
        z = zL

    Omz = cosmo.Om(z)
    OLz = 1 - Omz
    dc = 18 * pi**2 - 82 * OLz - 39 * OLz**2
    return dc

def ddc(c, dc, dvir):
    dc1 = (dvir / 3.) * c**3 / (np.log(1.+c) - c/(1.+c))
    return np.abs(dc1 - dc)

def cconv(c, dvir, dvirnew):
    if np.isscalar(c):
        dc = (dvir / 3.) * c**3 / (np.log(1.+c) - c/(1.+c))
        cnew = golden(ddc, args=(dc, dvirnew), brack=[1, 30])
    else:
        cnew = [cconv(c1, dvir, dvirnew) for c1 in c]
        cnew = np.array(cnew)
    return cnew

def dMvirrs(rsKPC, c, dvir, Mvir):
    NFW1 = NFW(zL)  # Use H0 in units of km/s/Mpc
    NFW1.set(rsKPC * u.kpc, c=c, dvir=dvir)
    Mvir1 = NFW1.Msph(NFW1.rvir).to(u.M_sun).value
    return np.abs(Mvir.to(u.M_sun).value - Mvir1)

class NFW:
    def __init__(self, z=0.2):
        global zL, DL, Ec, pc0, pc, p_E
        zL = z
        pc0 = (3 * H0**2) / (8 * pi * G)
        pc = (3 * cosmo.H(zL)**2) / (8 * pi * G)
        DL = cosmo.angular_diameter_distance(zL)
        Ec = c_light**2 / (4 * pi * G * DL)
        p_E = pc / Ec

    def test(self):
        print('DL:', DL.to(u.Mpc))
        print('c:', c_light.to(u.km / u.s))
        print('G:', G.to(u.m**3 / (u.kg * u.s**2)))
        print('Ec:', Ec.to(u.kg / u.m**2))
        print('H0:', H0.to(u.km / u.s / u.Mpc))
        print('pc0:', pc0.to(u.M_sun / u.kpc**3))
        print('p_E:', p_E.to(1/u.kpc))
        print('pc:', pc.to(u.M_sun / u.kpc**3))

    def set(self, rs=0*u.Mpc, c=None, dvir=200, rvir=0*u.Mpc, dc=None, ks=None, ps=0*u.kg/u.m**3, Mvir=0*u.M_sun):
        # define with any 2 parameters; rest will be calculated
        # additionally can set dvir or dc

        if Mvir > 0*u.M_sun and c and dvir:
            rs = golden(dMvirrs, args=(c, dvir, Mvir), brack=[1, 10000]) * u.kpc

        if ks:
            if rs.value == 0:
                rs = ks / p_E / dc
            elif dc is None:
                dc = ks / p_E / rs

        if ps > 0*u.kg/u.m**3:
            dc = ps / pc

        if c is None:
            if rs > 0*u.Mpc and rvir > 0*u.Mpc:
                c = rvir / rs
            elif dc:
                c = golden(ddc, args=(dc, dvir), brack=[1, 30])

        if dc is None:
            dc = (dvir / 3.) * c**3 / (np.log(1.+c) - c/(1.+c))

        if rs.value == 0:
            if rvir > 0*u.Mpc and c:
                rs = rvir / c

        if rvir.value == 0:
            rvir = rs * c

        if ks is None:
            ks = p_E * dc * rs

        if ps.value == 0:
            ps = dc * pc

        self.rs = rs
        self.c = c
        self.dvir = dvir
        self.rvir = rvir
        self.dc = dc
        self.ks = ks
        self.ps = ps

    def p(self, R):
        x = R / self.rs
        x = x.to(u.dimensionless_unscaled).value
        return self.ps / x / (1 + x)**2

    def F(self, x):
        Flo = (1 - np.arccosh(1/x) / np.sqrt(1 - x**2)) / (x**2 - 1)
        F1 = 1 / 3.
        Fhi = (1 - np.arccos(1/x) / np.sqrt(x**2 - 1)) / (x**2 - 1)
        Fall = np.where(x > 1, Fhi, Flo)
        Fall = np.where(x == 1, F1, Fall)
        return Fall

    def k(self, R, zs=None):
        x = R / self.rs
        x = x.to(u.dimensionless_unscaled).value
        k = 2 * self.ks * self.F(x)
        if zs is not None:
            k *= Dds_Ds(zL, zs)
        return k

    def y(self, R, zs=None):
        x = R / self.rs
        x = x.to(u.dimensionless_unscaled).value
        y = 2 * self.ks * (2 * self.Gcyl(x) / x**2 - self.F(x))
        if zs is not None:
            y *= Dds_Ds(zL, zs)
        return y

    def g(self, R, zs=None):
        return self.y(R, zs) / (1 - self.k(R, zs))

    def Gcyl(self, x):
        Glo = np.log(x / 2.) + np.arccosh(1/x) / np.sqrt(1 - x**2)
        G1  = np.log(x / 2.) + 1
        Ghi = np.log(x / 2.) + np.arccos(1/x) / np.sqrt(x**2 - 1)
        Gall = np.where(x > 1, Ghi, Glo)
        Gall = np.where(x == 1, G1, Gall)
        return Gall

    def Mcyl(self, R):
        x = R / self.rs
        x = x.to(u.dimensionless_unscaled).value
        M = 4 * pi * self.ks * self.Gcyl(x) * Ec * self.rs**2
        return M.to(u.M_sun)

    def kavgcyl(self, R, zs=1e30):
        x = R / self.rs
        x = x.to(u.dimensionless_unscaled).value
        Dds_Ds1 = Dds_Ds(zL, zs)
        return 4 * self.ks * self.Gcyl(x) / x**2 * Dds_Ds1

    def RE(self, xunit=u.kpc, zs=1e30):
        Dds_Ds1 = Dds_Ds(zL, zs)
        
        def dk(Rx):
            x = Rx * xunit / self.rs
            x = x.to(u.dimensionless_unscaled).value
            kavg = 4 * self.ks * self.Gcyl(x) / x**2 * Dds_Ds1
            return np.abs(kavg - 1)

        RE = golden(dk, brack=[1, 1000]) * xunit
        return RE

    def Gsph(self, x):
        return np.log(1 + x) - x / (1 + x)

    def Msph(self, R):
        x = R / self.rs
        x = x.to(u.dimensionless_unscaled).value
        M = 4 * pi * self.ps * self.rs**3 * self.Gsph(x)
        return M.to(u.M_sun)
    
    def Mvir(self):
        M = self.Msph(self.rvir)
        return M.to(u.M_sun)


In [4]:
z = 0.2  # lens redshift

In [5]:
dvir = dvir_calc(z)  # virial overdensity at that redshift
dvir

117.66888031293388

## Set the NFW halo parameters

2 parameters are required to define the rest.  
It's flexible, so you can choose which parameters you set.  
First, we'll do (M_vir, c)

In [6]:
halo = NFW()
halo.set(Mvir=1e15*u.M_sun, c=4, dvir=dvir)  # set (Mvir, c)

In [7]:
halo.Mvir()  # virial mass

<Quantity 9.99999994e+14 solMass>

In [8]:
rvir = halo.rvir.to(u.Mpc)  # virial radius
rvir

<Quantity 2.3048774 Mpc>

In [9]:
halo.Msph(rvir)  # mass within sphere

<Quantity 9.99999994e+14 solMass>

In [10]:
halo.Mcyl(rvir)  # mass within cylinder

/var/folders/y3/vsjr6j554mnfpgy_m1xrnckm0003tz/T/ipykernel_956/1019092455.py:135: RuntimeWarning: invalid value encountered in arccosh
  Glo = np.log(x / 2.) + np.arccosh(1/x) / np.sqrt(1 - x**2)
/var/folders/y3/vsjr6j554mnfpgy_m1xrnckm0003tz/T/ipykernel_956/1019092455.py:135: RuntimeWarning: invalid value encountered in sqrt
  Glo = np.log(x / 2.) + np.arccosh(1/x) / np.sqrt(1 - x**2)


<Quantity 1.2767913e+15 solMass>

Now try setting (r_vir, c)

In [11]:
halo.set(rvir=1*u.Mpc, c=4, dvir=dvir)  # set (rvir, c)
halo.Mvir()  # virial mass

<Quantity 8.16688625e+13 solMass>

## Set the NFW halo parameters

## intermediate quantities

In [12]:
halo.test()

DL: 680.6026992121936 Mpc
c: 299792.458 km / s
G: 6.6743e-11 m3 / (kg s2)
Ec: 5.102483462543831 kg / m2
H0: 70.0 km / (Mpc s)
pc0: 135.9929473504571 solMass / kpc3
p_E: 6.781558159875472e-08 1 / kpc
pc: 165.6938070517969 solMass / kpc3
